In [141]:
path = "/Users/alinazabolotskaya/Downloads/дискурс проверенные/"
example = "/Users/alinazabolotskaya/Downloads/дискурс проверенные/мария 8 июль/ксения 2 все т3/JK6441.cha"
excel_path = "/Users/alinazabolotskaya/Downloads/table20220121.xlsx"
import os
import pylangacq as pla
import pandas as pd

In [2]:
import nltk
from nltk.tokenize import RegexpTokenizer
nltk.download('punkt')
tokenizer = RegexpTokenizer(r'\w+')
import re

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/alinazabolotskaya/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### 0) Проверка всех файлов:
- какие надо допроверить (в названии оставляла пометку "check on mac")
- маркировка [% bad quality] в тех, где аудио повреждено/невозможно улучшить до нормального качества
- убрать Т3 из названия
- повторяющиеся файлы с маркировкой "было" занести в отдельный список

In [3]:
listing = []
repeated = []
for root, dirs, files in os.walk(path):
    for file in files:
        pathfile = os.path.join(root, file)
        if file.endswith('было.cha'):
            repeated.append((pathfile, file.strip(' было.cha')))
        elif file.endswith('.cha'):
            listing.append((pathfile, file.strip('.cha')))
            #print(file)
        #sx = pla.read_chat(pathfile)
        #with open(pathfile, 'r') as reader:
         #   f = reader.read()
          #  pauses = pauses(f)

In [4]:
ex = pla.read_chat(example)

Читаю все файлы, записываю в словарь.

In [26]:
def file_to_dict(textfile):
    dic = {}
    headers = []
    inv = []
    par = []
    cur = 0
    cnt = 0
    for line in textfile.split('\n'):
        if line.startswith('@'):
            headers.append(line)
            if line.startswith('@Media'):
                name = line.strip('@Media:\t').strip(', audio')
        elif line.startswith('*PAR'):
            cur = 'par'
            cnt += 1
            par.append( line)
        elif line.startswith('*INV'):
            inv.append(line)
            cnt += 1
            cur = 'inv'
        elif line.startswith('*') == False:
            if cur == 'par':
                ku = par[-1] + ' ' + line
                par = par[:-1]
                par.append(ku)
            if cur == 'inv':
                ku = inv[-1] + ' ' + line
                inv = inv[:-1]
                inv.append(ku)
    par_table = []
    for p in par:
        prep = p.split('\x15', 1)
        p_dic = {'n_utt': par.index(p) + 1, 
                'text_utt': prep[0].strip('*PAR:\t'),
                'time_utt': prep[-1].strip('\x15')}
        par_table.append(p_dic)

    dic.update({'name': name, 'headers': headers, 'par': par_table, 'inv': inv})
    return dic

In [195]:
def file_to_dict(textfile):
    dic = {}
    headers = []
    inv = []
    par = []
    cur = 0
    cnt = 0
    for line in textfile.split('\n'):
        if line.startswith('@'):
            headers.append(line)
            if line.startswith('@Media'):
                name = line.strip('@Media:\t').strip(', audio')
        elif line.startswith('*PAR'):
            cur = 'par'
            cnt += 1
            par.append({cnt: line})
        elif line.startswith('*INV'):
            cnt += 1
            cur = 'inv'
            inv.append({cnt: line})
        elif line.startswith('*') == False:
            if cur == 'par':
                ku = par[-1][cnt] + ' ' + line
                par = par[:-1]
                par.append({cnt: ku})
            if cur == 'inv':
                ku = inv[-1][cnt] + ' ' + line
                inv = inv[:-1]
                inv.append({cnt: ku})
    par_table = []
    for p in par:
        prep = list(p.values())[0].split('\x15', 1)
        p_dic = {'n_utt': list(p.keys())[0], 
                'text_utt': prep[0].strip('*PAR:\t'),
                'time_utt': prep[-1].strip('\x15')}
        par_table.append(p_dic)
    try:
        point = name.split('_')[1]
    except IndexError:
        point = "T1"
    dic.update({'name': name.split('_')[0], 'headers': headers, 'par': par_table, 'inv': inv, 'point': point})
    return dic

In [196]:
all_files = []
for file in listing:
    filename = file[1]
    pathfile = file[0]
    with open(pathfile, 'r') as reader:
        textfile = reader.read()
        
        all_files.append({filename: file_to_dict(textfile)})

In [197]:
all_files

[{'JX9879': {'name': 'JX9879',
   'headers': ['@UTF8',
    '@Window:\t0_0_0_0_0_0_579_0_579_0',
    '@Begin',
    '@Languages:\trus',
    '@Participants:\tPAR Participant, INV Investigator',
    '@ID:\trus|JX9879|PAR|||||Participant|||',
    '@ID:\trus|JX9879|INV|||||Investigator|||',
    '@Media:\tJX9879, audio',
    '@End'],
   'par': [{'n_utt': 1,
     'text_utt': 'хочу рассказать о поездке (0.279) на дачу (1.330) этим летом. ',
     'time_utt': '3200_10600'},
    {'n_utt': 2,
     'text_utt': 'погода стояла отвратительная. ',
     'time_utt': '12100_15550'},
    {'n_utt': 3,
     'text_utt': '&=uh &+с громыхал (1.185) гром. ',
     'time_utt': '18110_21391'},
    {'n_utt': 4, 'text_utt': 'сверкала молния. ', 'time_utt': '21391_23516'},
    {'n_utt': 5,
     'text_utt': 'лил (0.615) \tдождик что не видно было (0.238) дороги. ',
     'time_utt': '24073_28710'},
    {'n_utt': 6,
     'text_utt': 'ну вот но супруг не останавливался а все вел и вел машину.  ',
     'time_utt': '30300_34

In [198]:
dic = file_to_dict(textfile)

### 1) Создание датафрейма с паузами

__1.1. Определяем тип паузы__
- Паузы, рядом с которыми есть пометка &=er, &-это_самое и т. д. считаются заполненными.
- Паузы, рядом с которыми нет такой пометки - пустые.

Для поиска определенных паттернов текста (в данном случае число в скобках и пометка "&=") используется модуль regex.
В рассмотрение берем только участника *PAR.

__1.2 Заносим в словарь тип пауз, их длину, контекст, номер реплики и говорящего.__

In [199]:
def get_pauses(dic):
    pause_dic = []
    cnt = 0
    point = list(dic.values())[0]['point']
    filename = list(dic.values())[0]['name']
    for f in list(dic.values())[0]['par']:
        utt = f['n_utt']
        text_utt = f['text_utt']
        text_utt = re.sub(r'&-', '&=', text_utt, re.DOTALL) # привожу к одному формату все пометы-указания на заполненные паузы
        if utt - cnt == 1 and utt != 1:
            betw_pause = int(f['time_utt'].split('_')[0]) - int(end_prev)
            if text_utt.startswith('&='):
                betw_pause_type = 'filled'
            else:
                betw_pause_type = 'empty'
            pause_dic.append({'participant': filename, 'point': point, 'pause_type': betw_pause_type, 'pause_place': 'between_utt', 'pause_length': betw_pause * 0.001, 'context': text_prev + text_utt, 'utt_n': utt - 0.5})
        rawfilled = re.findall(r'\(\d+\.\d+\) &=\w+|&=\w+ \(\d+\.\d+\)', text_utt, re.DOTALL) # нахожу все вхождения заполненных пауз
        cnt = utt
        end_prev = f['time_utt'].split('_')[1]
        text_prev = text_utt
        for pause in rawfilled:
            pause = re.findall(r'\d+\.\d+', pause, re.DOTALL)[0]
            pause_dic.append({'participant': filename, 'point': point, 'pause_type': 'filled', 'pause_place': 'in_utt', 'pause_length': float(pause), 'context': text_utt, 'utt_n': utt})
        all_pauses = re.findall(r'\(\d+\.\d+\)', text_utt, re.DOTALL)
        for pause in all_pauses:
            pause = float(pause.strip(')').strip('('))
            if pause not in rawfilled:
                pause_dic.append({'participant': filename, 'point': point, 'pause_type': 'empty', 'pause_place': 'in_utt', 'pause_length': float(pause), 'context': text_utt, 'utt_n': utt})

    return pause_dic

In [200]:
all_pause_dic = []
cnt = 0
for file in all_files:
    all_pause_dic.extend(get_pauses(file))

__1.3 Создаем датафрейм__

In [201]:
pauses = pd.DataFrame(all_pause_dic)
pauses

,participant,point,pause_type,pause_place,pause_length,context,utt_n
0,JX9879,T1,empty,in_utt,0.279,хочу рассказать о поездке (0.279) на дачу (1.3...,1.0
1,JX9879,T1,empty,in_utt,1.330,хочу рассказать о поездке (0.279) на дачу (1.3...,1.0
2,JX9879,T1,empty,between_utt,1.500,хочу рассказать о поездке (0.279) на дачу (1.3...,1.5
3,JX9879,T1,filled,between_utt,2.560,погода стояла отвратительная. &=uh &+с громыха...,2.5
4,JX9879,T1,empty,in_utt,1.185,&=uh &+с громыхал (1.185) гром.,3.0
...,...,...,...,...,...,...,...
2825,JU5933,T1,empty,between_utt,0.887,своей внучке на (0.251) шестнадцать лет (0.405...,3.5
2826,JU5933,T1,empty,in_utt,0.267,в результате конечно сделала (0.267) очень кра...,4.0
2827,JU5933,T1,empty,between_utt,0.635,в результате конечно сделала (0.267) очень кра...,4.5
2828,JU5933,T1,filled,between_utt,0.173,со свечками. &=ну получился очень красивый и е...,5.5


__1.4 Чистка датафрейма__

Убираем все паузы длиной менее 0.200, если такие есть. 
(Этот шаг, возможно, можно еще доработать, надо придумать, как.)

In [202]:
pauses.shape

(2830, 7)

In [203]:
pauses = pauses[pauses.pause_length >= 0.2]
pauses.shape

(2585, 7)

### 2) Подсчет метрик.

In [233]:
pauses.groupby('participant').mean()['pause_length'] # Средняя длина паузы по участнику

participant
B6780     1.185429
BB5946    2.378722
BJ2008    1.066375
BU9305    0.562000
BZ3684    1.313186
            ...   
ZH6469    2.664500
ZH6769    0.942350
ZK9063    0.730450
ZU5403    1.011106
ZV8646    1.027083
Name: pause_length, Length: 87, dtype: float64

In [205]:
pauses.groupby(['participant', 'pause_type']).mean()['pause_length'] # Средняя длина пауз разных типов по участнику

participant  pause_type
B6780        empty         1.127026
             filled        1.944667
BB5946       empty         1.924583
             filled        3.287000
BJ2008       empty         0.997571
                             ...   
ZK9063       filled        1.031286
ZU5403       empty         0.901077
             filled        1.341192
ZV8646       empty         0.940294
             filled        1.237857
Name: pause_length, Length: 155, dtype: float64

In [204]:
# Pause rate (length of pauses of 1 speaker/number of utt
pauses.groupby('participant')['pause_length'].sum() / pauses.groupby('participant')['utt_n'].mean()

participant
B6780     8.570066
BB5946    6.881304
BJ2008    4.874857
BU9305    1.570807
BZ3684    5.477774
            ...   
ZH6469    3.856161
ZH6769    1.948010
ZK9063    2.623389
ZU5403    8.855158
ZV8646    2.777465
Length: 87, dtype: float64

In [140]:
# Pause per utterance (number of pauses of 1 speaker/number of utt
pauses.groupby('participant')['pause_length'].count()/ pauses.groupby('participant')['utt_n'].mean()

participant
B6780        7.229508
BB5946       2.892857
BJ2008       4.571429
BU9305       2.795031
BZ3684       4.171360
               ...   
ZH6769       2.067183
ZK9063       2.068085
ZK9063_T3    1.817204
ZU5403_T3    8.757895
ZV8646       2.704225
Length: 88, dtype: float64

### Подготовка к подсчету корреляций в точке 1. 

In [289]:
df = pauses.loc[pauses['point'] == 'T1']

In [290]:
excel = pd.read_excel(excel_path, index_col=0)  

In [291]:
from datetime import date

def calculate_age(born):
    today = date.today()
    try:
        return today.year - int(born.split('.')[2]) - ((today.month, today.day) < (int(born.split('.')[1]), int(born.split('.')[0])))
    except AttributeError:
        return None

In [292]:
excel['Возраст'] = excel['Дата рождения'].apply(lambda x: calculate_age(x))

In [293]:
moca_t1 = excel['Unnamed: 143']
mmse_t1 = excel['Unnamed: 102']
age = excel['Возраст']

In [294]:
df.shape, age.shape

((2239, 8), (471,))

In [295]:
moca_t1

Код пациента
NaN             МоСА
NaN              NaN
NaN       Общий балл
ОЕ1385            25
JY5933            23
             ...    
VP9941            27
LL0077            14
LI0743            24
ZG0021             4
AT2501            26
Name: Unnamed: 143, Length: 471, dtype: object

In [296]:
df = df.merge(excel[['Unnamed: 143']], left_on='participant', right_on='Код пациента', how='left')

In [297]:
df = df.merge(excel[['Unnamed: 102']], left_on='participant', right_on='Код пациента', how='left')

In [298]:
df = df.merge(excel[['Возраст']], left_on='participant', right_on='Код пациента', how='left')

In [299]:
df = df.rename(columns={'Unnamed: 143': 'moca', 'Unnamed: 102': 'mmse', 'Возраст': 'age'})
df

,participant,point,pause_type,pause_place,pause_length,context,utt_n,mean_pause_by_participant,moca,mmse,age
0,JX9879,T1,empty,in_utt,0.279,хочу рассказать о поездке (0.279) на дачу (1.3...,1.0,NaN,26,27,62.0
1,JX9879,T1,empty,in_utt,1.330,хочу рассказать о поездке (0.279) на дачу (1.3...,1.0,NaN,26,27,62.0
2,JX9879,T1,empty,between_utt,1.500,хочу рассказать о поездке (0.279) на дачу (1.3...,1.5,NaN,26,27,62.0
3,JX9879,T1,filled,between_utt,2.560,погода стояла отвратительная. &=uh &+с громыха...,2.5,NaN,26,27,62.0
4,JX9879,T1,empty,in_utt,1.185,&=uh &+с громыхал (1.185) гром.,3.0,NaN,26,27,62.0
...,...,...,...,...,...,...,...,...,...,...,...
2234,JU5933,T1,empty,in_utt,0.405,своей внучке на (0.251) шестнадцать лет (0.405...,3.0,NaN,NaN,NaN,NaN
2235,JU5933,T1,empty,between_utt,0.887,своей внучке на (0.251) шестнадцать лет (0.405...,3.5,NaN,NaN,NaN,NaN
2236,JU5933,T1,empty,in_utt,0.267,в результате конечно сделала (0.267) очень кра...,4.0,NaN,NaN,NaN,NaN
2237,JU5933,T1,empty,between_utt,0.635,в результате конечно сделала (0.267) очень кра...,4.5,NaN,NaN,NaN,NaN


In [300]:
df.dropna(subset=['mmse', 'moca'])

,participant,point,pause_type,pause_place,pause_length,context,utt_n,mean_pause_by_participant,moca,mmse,age
0,JX9879,T1,empty,in_utt,0.279,хочу рассказать о поездке (0.279) на дачу (1.3...,1.0,NaN,26,27,62.0
1,JX9879,T1,empty,in_utt,1.330,хочу рассказать о поездке (0.279) на дачу (1.3...,1.0,NaN,26,27,62.0
2,JX9879,T1,empty,between_utt,1.500,хочу рассказать о поездке (0.279) на дачу (1.3...,1.5,NaN,26,27,62.0
3,JX9879,T1,filled,between_utt,2.560,погода стояла отвратительная. &=uh &+с громыха...,2.5,NaN,26,27,62.0
4,JX9879,T1,empty,in_utt,1.185,&=uh &+с громыхал (1.185) гром.,3.0,NaN,26,27,62.0
...,...,...,...,...,...,...,...,...,...,...,...
2227,WU4008,T1,empty,between_utt,2.311,спрашивала у мамы какое сегодня чило и сколько...,8.5,NaN,23,27,72.0
2228,WU4008,T1,empty,between_utt,0.628,она заранее приготовила ей уголок куда бы она ...,9.5,NaN,23,27,72.0
2229,WU4008,T1,filled,in_utt,0.371,купить &=mm (0.371) шкафчик куда она будет веш...,11.0,NaN,23,27,72.0
2230,WU4008,T1,empty,in_utt,0.371,купить &=mm (0.371) шкафчик куда она будет веш...,11.0,NaN,23,27,72.0


In [301]:
df['mmse'] = df['mmse'].apply(lambda x: float(x))
df['moca'] = df['moca'].apply(lambda x: float(x))

In [319]:
from scipy.stats import pearsonr

def calculate_pvalues(df):
    df = df.dropna()._get_numeric_data()
    dfcols = pd.DataFrame(columns=df.columns)
    pvalues = dfcols.transpose().join(dfcols, how='outer')
    for r in df.columns:
        for c in df.columns:
            pvalues[r][c] = round(pearsonr(df[r], df[c])[1], 4)
    return pvalues

# Корреляции:

In [302]:
df.groupby('participant').mean().corr(method='pearson')
#Коэффициент корреляции Пирсона между средней длиной паузы и mmse: -0.254062
#Коэффициент корреляции Пирсона между средней длиной паузы и moca: -0.176292
#Коэффициент корреляции Пирсона между средней длиной паузы и возрастом: 0.148773

,pause_length,utt_n,mean_pause_by_participant,moca,mmse,age
pause_length,1.000000,0.094797,NaN,-0.176292,-0.254062,0.148773
utt_n,0.094797,1.000000,NaN,-0.148015,-0.128083,0.144854
mean_pause_by_participant,NaN,NaN,NaN,NaN,NaN,NaN
moca,-0.176292,-0.148015,NaN,1.000000,0.624750,-0.440050
mmse,-0.254062,-0.128083,NaN,0.624750,1.000000,-0.251227
age,0.148773,0.144854,NaN,-0.440050,-0.251227,1.000000


In [328]:
calculate_pvalues(df.groupby('participant').mean())
#P-value средней длины паузы и mmse: 0.0395
##P-value средней длины паузы и moca: 0.1568
#P-value средней длины паузы и возраста: 0.8225

,pause_length,utt_n,moca,mmse,age,pause_rate
pause_length,0,0.819,0.1568,0.0395,0.8225,0
utt_n,0.819,0,0.2356,0.3054,0.3183,0.1676
moca,0.1568,0.2356,0,0,0.0002,0.5512
mmse,0.0395,0.3054,0,0,0.0419,0.9417
age,0.8225,0.3183,0.0002,0.0419,0,0.0981
pause_rate,0,0.1676,0.5512,0.9417,0.0981,0


In [303]:
df.loc[df['pause_type'] == 'empty'].groupby('participant').mean().corr(method='pearson')
#Коэффициент корреляции Пирсона между средней длиной незаполненной паузы и mmse: -0.270230
#Коэффициент корреляции Пирсона между средней длиной незаполненной паузы и moca: -0.189789
#Коэффициент корреляции Пирсона между средней длиной незаполненной паузы и возрастом: 0.166481

,pause_length,utt_n,mean_pause_by_participant,moca,mmse,age
pause_length,1.000000,0.095885,NaN,-0.189789,-0.270230,0.166481
utt_n,0.095885,1.000000,NaN,-0.143310,-0.117483,0.147405
mean_pause_by_participant,NaN,NaN,NaN,NaN,NaN,NaN
moca,-0.189789,-0.143310,NaN,1.000000,0.624750,-0.440050
mmse,-0.270230,-0.117483,NaN,0.624750,1.000000,-0.251227
age,0.166481,0.147405,NaN,-0.440050,-0.251227,1.000000


In [329]:
calculate_pvalues(df.loc[df['pause_type'] == 'empty'].groupby('participant').mean())
#P-value средней длины незаполненной паузы и mmse: 0.0282
##P-value средней длины незаполненной паузы и moca: 0.1269
#P-value средней длины незаполненной паузы и возраста: 0.6988

,pause_length,utt_n,moca,mmse,age,pause_rate
pause_length,0,0.7154,0.1269,0.0282,0.6988,0
utt_n,0.7154,0,0.251,0.3475,0.3066,0.1614
moca,0.1269,0.251,0,0,0.0002,0.5512
mmse,0.0282,0.3475,0,0,0.0419,0.9417
age,0.6988,0.3066,0.0002,0.0419,0,0.0981
pause_rate,0,0.1614,0.5512,0.9417,0.0981,0


In [304]:
df.loc[df['pause_type'] == 'filled'].groupby('participant').mean().corr(method='pearson')
#Коэффициент корреляции Пирсона между средней длиной заполненной паузы и mmse: -0.313677
#Коэффициент корреляции Пирсона между средней длиной заполненной паузы и moca: -0.179320
#Коэффициент корреляции Пирсона между средней длиной заполненной паузы и возрастом: -0.116277

,pause_length,utt_n,mean_pause_by_participant,moca,mmse,age
pause_length,1.000000,0.151529,NaN,-0.179320,-0.313677,-0.116277
utt_n,0.151529,1.000000,NaN,-0.215107,-0.218037,0.196474
mean_pause_by_participant,NaN,NaN,NaN,NaN,NaN,NaN
moca,-0.179320,-0.215107,NaN,1.000000,0.627116,-0.422802
mmse,-0.313677,-0.218037,NaN,0.627116,1.000000,-0.244964
age,-0.116277,0.196474,NaN,-0.422802,-0.244964,1.000000


In [331]:
calculate_pvalues(df.loc[df['pause_type'] == 'filled'].groupby('participant').mean())
#P-value средней длины заполненной паузы и mmse: 0.0265
##P-value средней длины заполненной паузы и moca: 0.2127
#P-value средней длины заполненной паузы и возраста: 0.4213

,pause_length,utt_n,moca,mmse,age,pause_rate
pause_length,0,0.2523,0.2127,0.0265,0.4213,0.0038
utt_n,0.2523,0,0.1336,0.1282,0.1715,0.2134
moca,0.2127,0.1336,0,0,0.0022,0.7994
mmse,0.0265,0.1282,0,0,0.0864,0.7467
age,0.4213,0.1715,0.0022,0.0864,0,0.1973
pause_rate,0.0038,0.2134,0.7994,0.7467,0.1973,0


In [305]:
df.loc[df['pause_place'] == 'in_utt'].groupby('participant').mean().corr(method='pearson')
#Коэффициент корреляции Пирсона между средней длиной паузы внутри ЭДЕ и mmse: -0.070749
#Коэффициент корреляции Пирсона между средней длиной паузы внутри ЭДЕ и moca: -0.089933
#Коэффициент корреляции Пирсона между средней длиной паузы внутри ЭДЕ и возрастом: 0.024157

,pause_length,utt_n,mean_pause_by_participant,moca,mmse,age
pause_length,1.000000,0.085130,NaN,-0.089933,-0.070749,0.024157
utt_n,0.085130,1.000000,NaN,-0.118010,-0.114546,0.126562
mean_pause_by_participant,NaN,NaN,NaN,NaN,NaN,NaN
moca,-0.089933,-0.118010,NaN,1.000000,0.624750,-0.440050
mmse,-0.070749,-0.114546,NaN,0.624750,1.000000,-0.251227
age,0.024157,0.126562,NaN,-0.440050,-0.251227,1.000000


In [333]:
calculate_pvalues(df.loc[df['pause_place'] == 'in_utt'].groupby('participant').mean())
#P-value средней длины паузы внутри ЭДЕ и mmse: 0.5724
#P-value средней длины паузы внутри ЭДЕ и moca: 0.4727
#P-value средней длины паузы внутри ЭДЕ и возраста: 0.2819

,pause_length,utt_n,moca,mmse,age,pause_rate
pause_length,0,0.6125,0.4727,0.5724,0.2819,0
utt_n,0.6125,0,0.3453,0.3598,0.4121,0.1535
moca,0.4727,0.3453,0,0,0.0002,0.5512
mmse,0.5724,0.3598,0,0,0.0419,0.9417
age,0.2819,0.4121,0.0002,0.0419,0,0.0981
pause_rate,0,0.1535,0.5512,0.9417,0.0981,0


In [306]:
df.loc[df['pause_place'] == 'between_utt'].groupby('participant').mean().corr(method='pearson')
#Коэффициент корреляции Пирсона между средней длиной паузы между ЭДЕ и moca: -0.167532
#Коэффициент корреляции Пирсона между средней длиной паузы между ЭДЕ и mmse: -0.275518
#Коэффициент корреляции Пирсона между средней длиной паузы между ЭДЕ и возрастом: 0.143556

,pause_length,utt_n,mean_pause_by_participant,moca,mmse,age
pause_length,1.000000,0.047810,NaN,-0.167532,-0.275518,0.143556
utt_n,0.047810,1.000000,NaN,-0.226505,-0.189707,0.203617
mean_pause_by_participant,NaN,NaN,NaN,NaN,NaN,NaN
moca,-0.167532,-0.226505,NaN,1.000000,0.641687,-0.434836
mmse,-0.275518,-0.189707,NaN,0.641687,1.000000,-0.280733
age,0.143556,0.203617,NaN,-0.434836,-0.280733,1.000000


In [335]:
calculate_pvalues(df.loc[df['pause_place'] == 'between_utt'].groupby('participant').mean())
#P-value средней длины паузы между ЭДЕ и mmse: 0.0263
#P-value средней длины паузы между ЭДЕ и moca: 0.1822
#P-value средней длины паузы между ЭДЕ и возраста: 0.713

,pause_length,utt_n,moca,mmse,age,pause_rate
pause_length,0,0.7128,0.1822,0.0263,0.713,0
utt_n,0.7128,0,0.0696,0.1301,0.1296,0.11
moca,0.1822,0.0696,0,0,0.0003,0.6111
mmse,0.0263,0.1301,0,0,0.0235,0.8163
age,0.713,0.1296,0.0003,0.0235,0,0.143
pause_rate,0,0.11,0.6111,0.8163,0.143,0


In [307]:
pause_rate = df.groupby('participant')['pause_length'].sum() / df.groupby('participant')['utt_n'].mean()
#Pause rate (length of pauses of 1 speaker/number of utt

In [316]:
df = df.merge(pause_rate.rename('pause_rate'), left_on='participant', right_on='participant', how='left')

In [318]:
df.groupby('participant').mean().corr(method='pearson')
#Коэффициент корреляции Пирсона между pause_rate и moca: 0.074682
#Коэффициент корреляции Пирсона между pause_rate и mmse: 0.009184
#Коэффициент корреляции Пирсона между pause_rate и возрастом: -0.205923

,pause_length,utt_n,mean_pause_by_participant,moca,mmse,age,pause_rate
pause_length,1.000000,0.094797,NaN,-0.176292,-0.254062,0.148773,0.337664
utt_n,0.094797,1.000000,NaN,-0.148015,-0.128083,0.144854,-0.155519
mean_pause_by_participant,NaN,NaN,NaN,NaN,NaN,NaN,NaN
moca,-0.176292,-0.148015,NaN,1.000000,0.624750,-0.440050,0.074682
mmse,-0.254062,-0.128083,NaN,0.624750,1.000000,-0.251227,0.009184
age,0.148773,0.144854,NaN,-0.440050,-0.251227,1.000000,-0.205923
pause_rate,0.337664,-0.155519,NaN,0.074682,0.009184,-0.205923,1.000000


In [337]:
calculate_pvalues(df.groupby('participant').mean())
#P-value pause_rate и mmse: 0.9417	
#P-value pause_rate и moca: 0.5512
#P-value pause_rate и возраста: 0.0981

,pause_length,utt_n,moca,mmse,age,pause_rate
pause_length,0,0.819,0.1568,0.0395,0.8225,0
utt_n,0.819,0,0.2356,0.3054,0.3183,0.1676
moca,0.1568,0.2356,0,0,0.0002,0.5512
mmse,0.0395,0.3054,0,0,0.0419,0.9417
age,0.8225,0.3183,0.0002,0.0419,0,0.0981
pause_rate,0,0.1676,0.5512,0.9417,0.0981,0
